In [ ]:
import asyncio
from bleak import BleakScanner
import nest_asyncio
from dotenv import load_dotenv
import os

nest_asyncio.apply()
load_dotenv()

# RustコードにあるターゲットMACアドレス
# ※ 実際のデバイスに合わせて変更が必要な場合があります
TARGET_MAC_ADDRESS = os.getenv("CO2_METER_BLE_MAC_ADDRESS")

# SwitchBotのカンパニーID (0x0969)
COMPANY_ID = 2409


In [ ]:
def parse_switchbot_data(data: bytes):
    """
    Rustコードのバイト解析ロジックをPythonで再現した関数
    """
    # Rust: ((data[8] & 0x0f) as f32 * 0.1f32 + (data[9] & 0x7f) as f32)
    decimal_part = (data[8] & 0x0F) * 0.1
    integer_part = data[9] & 0x7F

    # Rust: * (if data[9] & 0x80 > 0 { 1f32 } else { -1f32 })
    # 0x80とのANDが正ならプラス、そうでなければマイナス
    sign = 1 if (data[9] & 0x80) > 0 else -1

    temp = (integer_part + decimal_part) * sign

    # Rust: data[10] & 0x7f
    humidity = data[10] & 0x7F

    # Rust: u16::from_be_bytes([data[13], data[14]])
    # big-endianでバイト列を数値に変換
    co2 = int.from_bytes(data[13:15], byteorder="big")

    return temp, humidity, co2


In [ ]:
async def main():
    print("Creating BLE manager and starting scan...")

    # スキャナーを開始
    async with BleakScanner() as scanner:
        print("Scanning... (Press Ctrl+C to stop)")

        while True:
            # Rustコードと同様に一定時間待機してからデータをチェック
            await asyncio.sleep(2.0)

            # 発見されたデバイスとアドバタイズデータを取得
            detected_devices = scanner.discovered_devices_and_advertisement_data

            # ターゲットのMACアドレスが発見済みリストにあるか確認
            # (Windows/LinuxはMACアドレス、macOSはUUIDで管理されるため注意)
            found_device_info = None

            for address, (device, advertisement_data) in detected_devices.items():
                if address.upper() == TARGET_MAC_ADDRESS.upper():
                    found_device_info = advertisement_data
                    break

            if not found_device_info:
                continue

            # Manufacturer Data (0x0969) を取得
            mfg_data = found_device_info.manufacturer_data.get(COMPANY_ID)

            if mfg_data:
                # データを解析
                temp, humidity, co2 = parse_switchbot_data(mfg_data)
                print(f"MacAddress:      {TARGET_MAC_ADDRESS}")
                print(f"Temperature:     {temp:.1f}℃")
                print(f"Humidity:        {humidity}%")
                print(f"CO2:             {co2}ppm")
                print("-" * 30)


In [ ]:
asyncio.run(main())